In [ ]:
# 주피터(Jupyter) 출력 자동 줄바꿈 + 보기 좋은 출력 유틸
from IPython.display import HTML, display
from pprint import pformat
import textwrap

display(HTML("""
<style>
/* JupyterLab 출력 줄바꿈 */
.jp-OutputArea-output pre,
.jp-RenderedText,
.jp-OutputArea-output {
    white-space: pre-wrap !important;
    word-break: break-word !important;
}

/* Classic Notebook 출력 줄바꿈 */
.output pre,
.text_cell_render pre {
    white-space: pre-wrap !important;
    word-break: break-word !important;
}
</style>
"""))


def pretty_print(obj, width=100, sort_dicts=False):
    """긴 객체(object)를 보기 좋게 줄바꿈 출력"""
    print(pformat(obj, width=width, sort_dicts=sort_dicts, compact=False))


def wrap_print(text, width=100):
    """긴 문자열(string)을 지정 폭으로 줄바꿈 출력"""
    print(
        textwrap.fill(
            str(text),
            width=width,
            replace_whitespace=False,
            drop_whitespace=False,
        )
    )


def print_choices(response, width=100):
    """OpenAI 응답의 choices를 보기 좋게 출력"""
    for idx, choice in enumerate(response.choices):
        print(f"[choice {idx}]")
        pretty_print(choice, width=width)
        print("-" * width)

In [ ]:
import os
import time
import openai
from dotenv import find_dotenv, load_dotenv

# 빠른 상태 점검(Health Check): 키 로딩 + API 왕복 시간 확인
dotenv_path = find_dotenv(usecwd=True)
load_dotenv(dotenv_path=dotenv_path)

api_key = os.getenv("OPENAI_API_KEY")
print("[1] OPENAI_API_KEY 존재 여부:", bool(api_key))
if not api_key:
    raise RuntimeError("OPENAI_API_KEY가 없습니다. .env를 확인하세요.")

client = openai.OpenAI(timeout=12.0, max_retries=0)

start = time.perf_counter()
try:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": "reply with pong"}],
    )
    elapsed = time.perf_counter() - start
    print(f"[2] API 호출 성공: {elapsed:.2f}초")
    print("[3] 응답:", response.choices[0].message.content)
except Exception as e:
    elapsed = time.perf_counter() - start
    print(f"[2] API 호출 실패: {elapsed:.2f}초")
    print("[3] 에러 타입:", type(e).__name__)
    print("[4] 에러 메시지:", str(e))

In [ ]:
import os
import openai
from dotenv import find_dotenv, load_dotenv

# Python 3.14 환경에서도 안정적으로 .env를 찾도록 명시적으로 경로를 지정
dotenv_path = find_dotenv(usecwd=True)
load_dotenv(dotenv_path=dotenv_path)

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("OPENAI_API_KEY가 비어 있습니다. .env 파일을 확인하세요.")

# 네트워크 상태가 나쁠 때 셀이 무한 대기처럼 보이지 않도록 timeout/retry를 설정
client = openai.OpenAI(timeout=20.0, max_retries=1)

PROMPT = """
I have the following functions in my system.

`get_weather`
`get_currency`
`get_news`

All of them receive the name of a country as an argumet (i.e get_news('Spain'))

Please answer with the name of the function that you would like me to run.

Please say nothing else, just the name of the function with the arguments.

Answer the following question:

What is the weather in Greece?
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": PROMPT}],
)

response

In [ ]:
message = response.choices[0].message.content
message

In [ ]:
for choice in response.choices:
    print (choice)

In [ ]:
message = response.choices[0].message.content
message

In [ ]:
import os
import openai
from dotenv import find_dotenv, load_dotenv

# 노트북 어디에서 실행해도 .env를 확실히 읽도록 고정
dotenv_path = find_dotenv(usecwd=True)
load_dotenv(dotenv_path=dotenv_path)

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("OPENAI_API_KEY가 비어 있습니다. .env 파일을 확인하세요.")

client = openai.OpenAI(timeout=20.0, max_retries=1)

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": "What is the capital of Greece?",
        }
    ],
)

response.choices[0].message.content


In [ ]:
# 긴 출력은 print_choices(response, width=100)로 확인
print_choices(response, width=100)

In [ ]:
message = response.choices[0].message.content

message

In [1]:
import os
import sys
from pathlib import Path

# 프로젝트 로컬 패키지 경로(.packages)를 우선 경로에 추가
PROJECT_ROOT = Path.cwd()
LOCAL_PACKAGES = PROJECT_ROOT / ".packages"
LOCAL_PACKAGES.mkdir(exist_ok=True)
if str(LOCAL_PACKAGES) not in sys.path:
    sys.path.insert(0, str(LOCAL_PACKAGES))

import openai


def load_openai_key_from_env_file():
    """python-dotenv 없이도 .env에서 OPENAI_API_KEY를 읽는다."""
    env_path = PROJECT_ROOT / ".env"
    if not env_path.exists():
        return None

    for raw_line in env_path.read_text(encoding="utf-8").splitlines():
        line = raw_line.strip()
        if not line or line.startswith("#") or "=" not in line:
            continue
        key, value = line.split("=", 1)
        if key.strip() == "OPENAI_API_KEY":
            return value.strip().strip('"').strip("'")
    return None


api_key = os.getenv("OPENAI_API_KEY") or load_openai_key_from_env_file()
if not api_key:
    raise RuntimeError("OPENAI_API_KEY를 찾지 못했습니다. .env 또는 환경변수를 확인하세요.")

client = openai.OpenAI(api_key=api_key, timeout=20.0, max_retries=1)
messages = []
print("초기화 완료: client, messages")


초기화 완료: client, messages


In [2]:
import json
import subprocess
from urllib import error, request


MOVIE_API_BASE_URL = "https://nomad-movies.nomadcoders.workers.dev"


def _fetch_json(path: str):
    """영화 API(Movie API)에서 JSON 응답을 가져옵니다.

    1) urllib 요청
    2) 403/네트워크 이슈 시 curl 폴백(Fallback)
    """
    url = f"{MOVIE_API_BASE_URL}{path}"

    try:
        req = request.Request(
            url=url,
            method="GET",
            headers={
                "User-Agent": "Mozilla/5.0",
                "Accept": "application/json",
            },
        )
        with request.urlopen(req, timeout=15) as resp:
            charset = resp.headers.get_content_charset() or "utf-8"
            return json.loads(resp.read().decode(charset))
    except Exception as first_error:
        try:
            # 일부 환경에서 urllib가 403을 받는 경우가 있어 curl로 재시도
            result = subprocess.run(
                [
                    "curl",
                    "-sS",
                    "--fail",
                    "--max-time",
                    "20",
                    "-H",
                    "User-Agent: Mozilla/5.0",
                    "-H",
                    "Accept: application/json",
                    url,
                ],
                check=True,
                capture_output=True,
                text=True,
            )
            return json.loads(result.stdout)
        except Exception as second_error:
            raise RuntimeError(
                f"Movie API 요청 실패: {url} | first={type(first_error).__name__}: {first_error} | "
                f"fallback={type(second_error).__name__}: {second_error}"
            ) from second_error


def get_popular_movies():
    """인기 영화 목록을 반환합니다."""
    return _fetch_json("/movies")


def get_movie_details(id: int):
    """영화 상세 정보를 반환합니다."""
    movie_id = int(id)
    return _fetch_json(f"/movies/{movie_id}")


def get_movie_credits(id: int):
    """영화 출연진/제작진 정보를 반환합니다."""
    movie_id = int(id)
    return _fetch_json(f"/movies/{movie_id}/credits")


def compact_tool_result(tool_name: str, data):
    """도구 응답 크기를 줄여 모델이 안정적으로 후속 답변하도록 돕습니다."""
    if tool_name == "get_popular_movies" and isinstance(data, list):
        return [
            {
                "id": m.get("id"),
                "title": m.get("title"),
                "release_date": m.get("release_date"),
                "vote_average": m.get("vote_average"),
            }
            for m in data[:10]
        ]

    if tool_name == "get_movie_details" and isinstance(data, dict):
        genres = data.get("genres") or []
        return {
            "id": data.get("id"),
            "title": data.get("title"),
            "original_title": data.get("original_title"),
            "release_date": data.get("release_date"),
            "runtime": data.get("runtime"),
            "vote_average": data.get("vote_average"),
            "overview": data.get("overview"),
            "genres": [g.get("name") for g in genres if isinstance(g, dict)],
        }

    if tool_name == "get_movie_credits" and isinstance(data, list):
        return [
            {
                "name": c.get("name"),
                "character": c.get("character"),
                "known_for_department": c.get("known_for_department"),
                "order": c.get("order"),
            }
            for c in data[:15]
        ]

    return data


MOVIE_TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_popular_movies",
            "description": "Get popular movies from /movies endpoint.",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_movie_details",
            "description": "Get movie details from /movies/:id endpoint.",
            "parameters": {
                "type": "object",
                "properties": {
                    "id": {
                        "type": "integer",
                        "description": "Movie ID",
                    }
                },
                "required": ["id"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_movie_credits",
            "description": "Get cast and crew from /movies/:id/credits endpoint.",
            "parameters": {
                "type": "object",
                "properties": {
                    "id": {
                        "type": "integer",
                        "description": "Movie ID",
                    }
                },
                "required": ["id"],
            },
        },
    },
]


MOVIE_SYSTEM_PROMPT = """
당신은 영화 전문가 에이전트(Movie Expert Agent)입니다.
아래 함수(Function)만 사용해서 답변하세요.

1) get_popular_movies(): /movies
2) get_movie_details(id): /movies/:id
3) get_movie_credits(id): /movies/:id/credits

규칙:
- 사용자의 요청을 해결할 때 필요한 함수(Function)가 있으면 반드시 tool call을 사용하세요.
- 함수 이름과 인자(arguments)는 정확히 선택하세요.
- 최종 답변은 한국어로 간결하게 작성하세요.
- 알 수 없는 정보는 추측하지 마세요.
""".strip()


MOVIE_FUNCTION_MAP = {
    "get_popular_movies": get_popular_movies,
    "get_movie_details": get_movie_details,
    "get_movie_credits": get_movie_credits,
}

print("Movie Expert Agent 초기화 완료")

Movie Expert Agent 초기화 완료


In [3]:
def run_movie_expert_agent(user_input: str, execute_tool: bool = True, max_steps: int = 4):
    """함수 호출(Function Calling) 선택을 확인하고, 필요 시 실제 API까지 실행합니다."""
    messages_local = [
        {"role": "system", "content": MOVIE_SYSTEM_PROMPT},
        {"role": "user", "content": user_input},
    ]

    printed_tool_header = False

    for _ in range(max_steps):
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages_local,
            tools=MOVIE_TOOLS,
            tool_choice="auto",
        )
        assistant_message = response.choices[0].message

        if assistant_message.tool_calls and execute_tool:
            if not printed_tool_header:
                print("[모델 함수 선택 결과]")
                printed_tool_header = True

            messages_local.append(
                {
                    "role": "assistant",
                    "content": assistant_message.content or "",
                    "tool_calls": [
                        {
                            "id": tc.id,
                            "type": tc.type,
                            "function": {
                                "name": tc.function.name,
                                "arguments": tc.function.arguments,
                            },
                        }
                        for tc in assistant_message.tool_calls
                    ],
                }
            )

            for tc in assistant_message.tool_calls:
                fn_name = tc.function.name
                args = json.loads(tc.function.arguments or "{}")
                print(f"- function: {fn_name}")
                print(f"  arguments: {args}")

                func = MOVIE_FUNCTION_MAP.get(fn_name)
                if func is None:
                    tool_result = {
                        "error": "UnknownFunction",
                        "message": f"지원하지 않는 함수입니다: {fn_name}",
                    }
                else:
                    try:
                        raw_result = func(**args)
                        tool_result = compact_tool_result(fn_name, raw_result)
                    except error.HTTPError as http_e:
                        tool_result = {
                            "error": f"HTTPError: {http_e.code}",
                            "message": str(http_e),
                            "tool": fn_name,
                        }
                        print(f"[도구 실행 오류] {fn_name}: HTTPError {http_e.code} - {http_e}")
                    except Exception as e:
                        tool_result = {
                            "error": type(e).__name__,
                            "message": str(e),
                            "tool": fn_name,
                        }
                        print(f"[도구 실행 오류] {fn_name}: {type(e).__name__} - {e}")

                if isinstance(tool_result, dict) and tool_result.get("error"):
                    print(f"[도구 실패 상세] {tool_result}")

                messages_local.append(
                    {
                        "role": "tool",
                        "tool_call_id": tc.id,
                        "content": json.dumps(tool_result, ensure_ascii=False),
                    }
                )

            continue

        final_text = assistant_message.content or "(빈 응답)"
        if not printed_tool_header:
            print("[모델 함수 선택 결과] tool_calls 없음")
        print("[최종 답변]")
        print(final_text)

        return {
            "final_answer": final_text,
        }

    fallback = "응답 생성 단계가 너무 길어져 중단했습니다. 다시 시도해 주세요."
    print("[최종 답변]")
    print(fallback)
    return {"final_answer": fallback}


In [5]:
print("터미널 입력 모드 시작")
print("질문을 입력하면 영화 에이전트가 답변합니다.")
print("종료하려면 'exit', 'quit', '종료' 중 하나를 입력하세요.")

while True:
    user_input = input("\n질문 입력 > ").strip()

    if user_input.lower() in {"exit", "quit"} or user_input == "종료":
        print("입력 모드를 종료합니다.")
        break

    if not user_input:
        print("질문이 비어 있습니다. 다시 입력해 주세요.")
        continue

    print("=" * 100)
    print(f"[사용자 입력] {user_input}")
    run_movie_expert_agent(user_input=user_input, execute_tool=True)
    print()

[테스트 1] 사용자 입력: 지금 인기 있는 영화가 무엇인지 알려줘
[모델 함수 선택 결과]
- function: get_popular_movies
  arguments: {}
[최종 답변]
현재 인기 있는 영화는 다음과 같습니다:

1. **Mercy**
   - 개봉일: 2026-01-20
   - 평점: 7.1

2. **28 Years Later: The Bone Temple**
   - 개봉일: 2026-01-14
   - 평점: 7.199

3. **The Orphans**
   - 개봉일: 2025-08-20
   - 평점: 6.043

4. **A Woman Scorned**
   - 개봉일: 2025-06-09
   - 평점: 6

5. **Deathstalker**
   - 개봉일: 2025-10-10
   - 평점: 6.29

6. **Greenland 2: Migration**
   - 개봉일: 2026-01-07
   - 평점: 6.517

7. **Your Heart Will Be Broken**
   - 개봉일: 2026-03-26
   - 평점: 0

8. **Space/Time**
   - 개봉일: 2025-05-01
   - 평점: 5.859

9. **The Shadow's Edge**
   - 개봉일: 2025-08-16
   - 평점: 7.241

10. **Zootopia 2**
    - 개봉일: 2025-11-26
    - 평점: 7.615

관심 있는 영화가 있으면 더 자세히 알려드릴 수 있습니다.

[테스트 2] 사용자 입력: movie ID 550에 해당하는 영화가 무엇인지 알려줘
[모델 함수 선택 결과]
- function: get_movie_details
  arguments: {'id': 550}
[최종 답변]
영화 ID 550은 "파이터 클럽(Fight Club)"입니다. 1999년 10월 15일에 개봉하였으며, 장르는 드라마와 스릴러입니다. 이 영화는 불면증에 시달리는 남자와 비밀리에 비누를 판매하는 